## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-07-05-07-51 +0000,bbc,Baby among two killed in Russian drone attack ...,https://www.bbc.com/news/articles/cpq5dl5y4nlo...
1,2025-09-07-05-01-27 +0000,nypost,Stephen King predicts Trump supporters will de...,https://nypost.com/2025/09/07/us-news/stephen-...
2,2025-09-07-04-59-51 +0000,nypost,US teen pilot Ethan Guo released from Antarcti...,https://nypost.com/2025/09/07/world-news/us-te...
3,2025-09-07-04-47-35 +0000,startribune,Review: Patriotic Jason Aldean waves his small...,https://www.startribune.com/jason-aldean-grand...
4,2025-09-07-04-46-59 +0000,wapo,ICE begins immigration crackdown in Massachuse...,https://www.washingtonpost.com/nation/2025/09/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2380/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
11,trump,23
42,immigration,10
69,raid,9
64,president,8
2,two,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
20,2025-09-07-03-00-00 +0000,wsj,When President Trump unveiled his EU trade dea...,https://www.wsj.com/economy/trade/the-eus-trad...,72
7,2025-09-07-04-24-23 +0000,nypost,South Korean President vows support to over 30...,https://nypost.com/2025/09/07/world-news/south...,57
35,2025-09-07-01-00-00 +0000,wsj,"Trump has so far stood by RFK Jr., the health ...",https://www.wsj.com/politics/policy/trump-said...,55
173,2025-09-06-09-00-00 +0000,wsj,Rep. Marjorie Taylor Greene is walking a tight...,https://www.wsj.com/politics/policy/marjorie-t...,54
1,2025-09-07-05-01-27 +0000,nypost,Stephen King predicts Trump supporters will de...,https://nypost.com/2025/09/07/us-news/stephen-...,50


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
20,72,2025-09-07-03-00-00 +0000,wsj,When President Trump unveiled his EU trade dea...,https://www.wsj.com/economy/trade/the-eus-trad...
7,49,2025-09-07-04-24-23 +0000,nypost,South Korean President vows support to over 30...,https://nypost.com/2025/09/07/world-news/south...
35,32,2025-09-07-01-00-00 +0000,wsj,"Trump has so far stood by RFK Jr., the health ...",https://www.wsj.com/politics/policy/trump-said...
21,31,2025-09-07-02-52-37 +0000,nypost,"Mamdani, Sanders use Brooklyn College event to...",https://nypost.com/2025/09/06/us-news/mamdani-...
1,27,2025-09-07-05-01-27 +0000,nypost,Stephen King predicts Trump supporters will de...,https://nypost.com/2025/09/07/us-news/stephen-...
52,26,2025-09-06-22-20-46 +0000,nyt,Authorities Point to Cable Disconnecting in Fi...,https://www.nytimes.com/2025/09/06/world/europ...
49,26,2025-09-06-22-46-40 +0000,nypost,Cheerleader Laken Snelling’s ex-boyfriend prov...,https://nypost.com/2025/09/06/us-news/universi...
45,26,2025-09-06-23-27-00 +0000,wsj,Air Canada Flight Attendants Reject Tentative ...,https://www.wsj.com/business/airlines/air-cana...
63,25,2025-09-06-20-59-15 +0000,nypost,"Israel issues evacuation orders, tells Palesti...",https://nypost.com/2025/09/06/world-news/israe...
85,22,2025-09-06-18-06-05 +0000,bbc,Pilot who tried to cut off engine midair after...,https://www.bbc.com/news/articles/c4gq9n6kwxeo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
